In [17]:
# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, transpile
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from ibm_quantum_widgets import *
from qiskit_aer import AerSimulator

# qiskit-ibmq-provider has been deprecated.
# Please see the Migration Guides in https://ibm.biz/provider_migration_guide for more detail.
from qiskit_ibm_runtime import QiskitRuntimeService, Sampler, Estimator, Session, Options

# Loading your IBM Quantum account(s)
service = QiskitRuntimeService(channel="ibm_quantum")

# Invoke a primitive inside a session. For more details see https://qiskit.org/documentation/partners/qiskit_ibm_runtime/tutorials.html
# with Session(backend=service.backend("ibmq_qasm_simulator")):
#     result = Sampler().run(circuits).result()

In [30]:
import torch
import torch.nn as nn
num_epochs = 10 
class MyNet(nn.Module):
    def __init__(self):
        super(MyNet, self).__init__()
        self.fc1 = nn.Linear(784, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 10)

    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = nn.functional.relu(self.fc1(x))
        x = nn.functional.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
model = MyNet()

optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
criterion = nn.CrossEntropyLoss()


for epoch in range(num_epochs):
    for i, (inputs, targets) in enumerate(data_loader):
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        if i % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(
                epoch, num_epochs, len(data_loader), loss.item()))
            
with torch.no_grad():
    correct = 0
    total = 0
    for inputs, targets in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += targets.size(0)
        correct += (predicted == targets).sum().item()

    print('Accuracy of the model on the test images: {} %'.format(100 * correct / total))


Traceback (most recent call last):
  Cell In[30], line 24
    for epoch in range(num_epochs):
TypeError: range() missing 1 required positional arguments: "end"

Use %tb to get the full traceback.


In [13]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
criterion = nn.CrossEntropyLoss()

for epoch in range(num_epochs):
    for i, (inputs, targets) in enumerate(data_loader):
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        if i % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(
                epoch+1, num_epochs, i+1, len(data_loader), loss.item()))


Traceback (most recent call last):
  Cell In[13], line 1
    optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
NameError: name 'torch' is not defined

Use %tb to get the full traceback.


In [33]:
import torch.optim as optim
import torch.nn as nn
num_epochs = 10

class MyNet(nn.Module):
    def __init__(self):
        super(MyNet, self).__init__()
        self.fc1 = nn.Linear(784, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 10)

    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = nn.functional.relu(self.fc1(x))
        x = nn.functional.relu(self.fc2(x))
        x = self.fc3(x)
        return x
class CustomOptimizer(optim.Optimizer):
    def __init__(self, params, lr=0.01, momentum=0.9, weight_decay=0.0, d_dim=0, k=0, alpha=0.0):
        self.lr = lr
        self.momentum = momentum
        self.weight_decay = weight_decay
        self.d_dim = d_dim
        self.k = k
        self.alpha = alpha
        defaults = dict(lr=lr, momentum=momentum, weight_decay=weight_decay, d_dim=d_dim, k=k, alpha=alpha)
        super(CustomOptimizer, self).__init__(params, defaults)

    def step(self):
        for group in self.param_groups:
            for p in group['params']:
                if p.grad is None:
                    continue
                grad = p.grad.data
                shape = grad.shape
                d_dim = group['d_dim']
                k = group['k']
                alpha = group['alpha']
                flatten_dim = 1
                for i in range(len(shape)):
                    if shape[i] == d_dim:
                        flatten_dim = i
                G = grad.view(-1, shape[flatten_dim])
                U, S, V = torch.svd(G)
                Wk = torch.mm(U[:, :k], torch.mm(torch.diag(S[:k]), V[:k, :]))
                W2k = torch.mm(U[:, :2*k], torch.mm(torch.diag(S[:2*k]), V[:2*k, :]))
                fnorm = torch.norm(W2k, p='fro')
                if 'momentum_buffer' not in state:
                    state['momentum_buffer'] = torch.ones(1).to(p.device)
                Gk = Wk / (fnorm / state['momentum_buffer'])
                top_alpha_percentile = torch.kthvalue(torch.abs(Gk).view(-1), int((1 - alpha) * Gk.numel()))[0]
                Gk = torch.where(torch.abs(Gk) >= top_alpha_percentile, Gk, torch.zeros_like(Gk))
                if 'momentum_buffer' not in state:
                    state['momentum_buffer'] = torch.zeros_like(Gk)
                state['momentum_buffer'] = state['momentum_buffer'] * self.momentum + Gk
                p.data.add_(-group['lr'], state['momentum_buffer'])
                if group['weight_decay'] != 0:
                   p.data.add_(-group['weight_decay'], p.data)

model = MyNet()  # Replace with an instance of your actual PyTorch model class
optimizer = CustomOptimizer(model.parameters(), lr=0.1, momentum=0.9, weight_decay=0.0001, d_dim=3, k=10, alpha=0.2)
criterion = nn.CrossEntropyLoss()

# Train the model
for epoch in range(num_epochs):
    for inputs, targets in data_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

# Print the results
print(optimizer.lr)
print(optimizer.momentum)
print(optimizer.weight_decay)
print(optimizer.d_dim)
print(optimizer.k)
print(optimizer.alpha)


Traceback (most recent call last):
  Cell In[33], line 65
    for epoch in range(num_epochs):
TypeError: range() missing 1 required positional arguments: "end"

Use %tb to get the full traceback.
